# Advanced House Price Prediction Using Pytorch

PyTorch, an open-source machine learning library, offers dynamic computation, enabling flexible model building and experimentation. Its tensor-based operations and automatic differentiation make it popular for research and production, empowering developers to create efficient and innovative deep learning models with ease.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/houseprice.csv')

In [ ]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
# these are the useful columns for our price prediction
df = pd.read_csv('/content/houseprice.csv', usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   object 
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   object 
 5   LotShape     1201 non-null   object 
 6   YearBuilt    1201 non-null   int64  
 7   1stFlrSF     1201 non-null   int64  
 8   2ndFlrSF     1201 non-null   int64  
 9   SalePrice    1201 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 103.2+ KB


In [ ]:
df.shape

(1201, 10)

In [ ]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF,SalePrice
0,60,RL,65.0,8450,Pave,Reg,2003,856,854,208500
1,20,RL,80.0,9600,Pave,Reg,1976,1262,0,181500
2,60,RL,68.0,11250,Pave,IR1,2001,920,866,223500
3,70,RL,60.0,9550,Pave,IR1,1915,961,756,140000
4,60,RL,84.0,14260,Pave,IR1,2000,1145,1053,250000


In [ ]:
for i in df.columns:
  print("Column name --{}-- and their unique values are ::== {}".format(i, len(df[i].unique())))

Column name --MSSubClass-- and their unique values are ::== 15
Column name --MSZoning-- and their unique values are ::== 5
Column name --LotFrontage-- and their unique values are ::== 110
Column name --LotArea-- and their unique values are ::== 869
Column name --Street-- and their unique values are ::== 2
Column name --LotShape-- and their unique values are ::== 4
Column name --YearBuilt-- and their unique values are ::== 112
Column name --1stFlrSF-- and their unique values are ::== 678
Column name --2ndFlrSF-- and their unique values are ::== 368
Column name --SalePrice-- and their unique values are ::== 597


In [ ]:
df['YearBuilt'].head(8)
# len(df['YearBuilt'].unique())

0    2003
1    1976
2    2001
3    1915
4    2000
5    1993
6    2004
8    1931
Name: YearBuilt, dtype: int64

In [ ]:
import datetime
datetime.datetime.now().year

2023

In [ ]:
# We are calculating the diff. year for predicting the price
df['Total Years'] = datetime.datetime.now().year - df['YearBuilt']

In [ ]:
df.drop('YearBuilt', axis = 1, inplace=True)

In [ ]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total Years
0,60,RL,65.0,8450,Pave,Reg,856,854,208500,20
1,20,RL,80.0,9600,Pave,Reg,1262,0,181500,47
2,60,RL,68.0,11250,Pave,IR1,920,866,223500,22
3,70,RL,60.0,9550,Pave,IR1,961,756,140000,108
4,60,RL,84.0,14260,Pave,IR1,1145,1053,250000,23


In [ ]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', '1stFlrSF', '2ndFlrSF', 'SalePrice', 'Total Years'],
      dtype='object')

In [ ]:
for i in df.columns:
  print("Column name --{}-- and their unique values are ::== {}".format(i, len(df[i].unique())))

Column name --MSSubClass-- and their unique values are ::== 15
Column name --MSZoning-- and their unique values are ::== 5
Column name --LotFrontage-- and their unique values are ::== 110
Column name --LotArea-- and their unique values are ::== 869
Column name --Street-- and their unique values are ::== 2
Column name --LotShape-- and their unique values are ::== 4
Column name --1stFlrSF-- and their unique values are ::== 678
Column name --2ndFlrSF-- and their unique values are ::== 368
Column name --SalePrice-- and their unique values are ::== 597
Column name --Total Years-- and their unique values are ::== 112


In [ ]:
# Here by analyzing the dataset, who has less unique values, we are extracting as Categorical Features

cate_feature = ['MSSubClass', 'MSZoning', 'Street', 'LotShape']
out_feature = 'SalePrice'

In [ ]:
# We are encoding the categorical features into integer type to use to train the Torch NN
# We are using LabelEncoder here, not One Hot Encoder
from sklearn.preprocessing import LabelEncoder

lbl_encoder = {}
lbl_encoder['MSSubClass'] = LabelEncoder()
lbl_encoder['MSSubClass'].fit_transform(df['MSSubClass'])

array([5, 0, 5, ..., 6, 0, 0])

In [ ]:
lbl_encoder

{'MSSubClass': LabelEncoder()}

In [ ]:
# We are encoding all the Categorical features columns
from sklearn.preprocessing import LabelEncoder

lbl_encoders = {}

for i in cate_feature:
  lbl_encoders[i] = LabelEncoder()
  df[i] = lbl_encoders[i].fit_transform(df[i])

In [ ]:
df.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total Years
0,5,3,65.0,8450,1,3,856,854,208500,20
1,0,3,80.0,9600,1,3,1262,0,181500,47
2,5,3,68.0,11250,1,0,920,866,223500,22
3,6,3,60.0,9550,1,0,961,756,140000,108
4,5,3,84.0,14260,1,0,1145,1053,250000,23
5,4,3,85.0,14115,1,0,796,566,143000,30
6,0,3,75.0,10084,1,3,1694,0,307000,19
8,4,4,51.0,6120,1,3,1022,752,129900,92
9,14,3,50.0,7420,1,3,1077,0,118000,84
10,0,3,70.0,11200,1,3,1040,0,129500,58


In [ ]:
# combining all the categorical features at one array
import numpy as np
# cate_feature = np.stack([df['MSSubClass'], df['MSZoning'], df['LotShape'], df['Street']], 1)
cate_feature=np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)

cate_feature

array([[5, 3, 1, 3],
       [0, 3, 1, 3],
       [5, 3, 1, 0],
       ...,
       [6, 3, 1, 3],
       [0, 3, 1, 3],
       [0, 3, 1, 3]])

In [ ]:
import torch

cate_feature = torch.tensor(cate_feature, dtype = torch.int64)

cate_feature


tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [ ]:
cont_features = []

for i in df.columns:
  if i in ["MSSubClass", "MSZoning", "Street", "LotShape","SalePrice"]:
    pass

  else:
    cont_features.append(i)

In [ ]:
cont_features

['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'Total Years']

In [ ]:
# combining all the continuous features at one array and converting to Tensor for training the model

cont_values = np.stack([df[i].values for i in cont_features], axis = 1)
cont_values = torch.tensor(cont_values, dtype = torch.float)

cont_values

tensor([[   65.,  8450.,   856.,   854.,    20.],
        [   80.,  9600.,  1262.,     0.,    47.],
        [   68., 11250.,   920.,   866.,    22.],
        ...,
        [   66.,  9042.,  1188.,  1152.,    82.],
        [   68.,  9717.,  1078.,     0.,    73.],
        [   75.,  9937.,  1256.,     0.,    58.]])

In [ ]:
cont_values.dtype

torch.float32

In [ ]:
y = torch.tensor(df['SalePrice'].values, dtype = torch.float).reshape(-1, 1)

y[15:21]

tensor([[139000.],
        [325300.],
        [139400.],
        [230000.],
        [129900.],
        [256300.]])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   int64  
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   int64  
 5   LotShape     1201 non-null   int64  
 6   1stFlrSF     1201 non-null   int64  
 7   2ndFlrSF     1201 non-null   int64  
 8   SalePrice    1201 non-null   int64  
 9   Total Years  1201 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 103.2 KB


In [ ]:
cate_feature.shape, cont_values.shape, y.shape

(torch.Size([1201, 4]), torch.Size([1201, 5]), torch.Size([1201, 1]))

In [ ]:
# Embedding size for Categorical Columns
cat_dims=[len(df[col].unique()) for col in ["MSSubClass", "MSZoning", "Street", "LotShape"]]

In [ ]:
cat_dims

[15, 5, 2, 4]

In [ ]:
# Thumbs Rule : Output dimension should be setbased on the input dimension(min(50,feature dimension/ 2))

embedding_dims = [(x, min(50, (x+1) // 2)) for x in cat_dims]

embedding_dims

[(15, 8), (5, 3), (2, 1), (4, 2)]

In [ ]:
# Embedding all the input and all the Output layers:
# Ex: 15 inputs give 8 outputs

# ModuleList : It is commonly used to store and manage a collection of layers or modules in a neural network.

# Embedding : is a PyTorch module that represents an embedding layer. An embedding layer is used to convert categorical variables (like indices or labels)
# into continuous-valued representations (embeddings) that can be learned during training.

import torch
import torch.nn as nn
import torch.nn.functional as F

embed_rep = nn.ModuleList([nn.Embedding(inp, out) for inp, out in embedding_dims])
embed_rep

ModuleList(
  (0): Embedding(15, 8)
  (1): Embedding(5, 3)
  (2): Embedding(2, 1)
  (3): Embedding(4, 2)
)

In [ ]:
cate_4_z = cate_feature[:4]
cate_4_z
# taking 4 starting points for Embedding

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        [6, 3, 1, 0]])

In [ ]:
pd.set_option('display.max_rows', 400)

embedding_val = []

for i, e in enumerate(embed_rep):
    embedding_val.append(e(cate_feature[:,i]))

In [ ]:
embedding_val

[tensor([[ 1.4458,  0.4582, -0.6858,  ...,  0.2635,  0.1580, -0.4048],
         [ 0.1650,  1.0161,  0.3631,  ...,  0.9128,  1.6083,  0.3455],
         [ 1.4458,  0.4582, -0.6858,  ...,  0.2635,  0.1580, -0.4048],
         ...,
         [ 0.3462,  0.7570, -0.8270,  ...,  1.0515, -1.3614,  1.7168],
         [ 0.1650,  1.0161,  0.3631,  ...,  0.9128,  1.6083,  0.3455],
         [ 0.1650,  1.0161,  0.3631,  ...,  0.9128,  1.6083,  0.3455]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[-0.5248,  1.2216,  0.9923],
         [-0.5248,  1.2216,  0.9923],
         [-0.5248,  1.2216,  0.9923],
         ...,
         [-0.5248,  1.2216,  0.9923],
         [-0.5248,  1.2216,  0.9923],
         [-0.5248,  1.2216,  0.9923]], grad_fn=<EmbeddingBackward0>),
 tensor([[-0.3540],
         [-0.3540],
         [-0.3540],
         ...,
         [-0.3540],
         [-0.3540],
         [-0.3540]], grad_fn=<EmbeddingBackward0>),
 tensor([[0.8949, 0.8230],
         [0.8949, 0.8230],
         [0.2027, 0.5680],

In [ ]:
z = torch.cat(embedding_val, 1)
# Concatenation in column wise
z

tensor([[ 1.4458,  0.4582, -0.6858,  ..., -0.3540,  0.8949,  0.8230],
        [ 0.1650,  1.0161,  0.3631,  ..., -0.3540,  0.8949,  0.8230],
        [ 1.4458,  0.4582, -0.6858,  ..., -0.3540,  0.2027,  0.5680],
        ...,
        [ 0.3462,  0.7570, -0.8270,  ..., -0.3540,  0.8949,  0.8230],
        [ 0.1650,  1.0161,  0.3631,  ..., -0.3540,  0.8949,  0.8230],
        [ 0.1650,  1.0161,  0.3631,  ..., -0.3540,  0.8949,  0.8230]],
       grad_fn=<CatBackward0>)

In [ ]:
dropout = nn.Dropout(0.4)
# dropping out 40% of data in NN
# dropout

In [ ]:
final_embed=dropout(z)
final_embed

tensor([[ 2.4096,  0.7637, -1.1430,  ..., -0.5900,  1.4915,  1.3716],
        [ 0.2750,  0.0000,  0.6051,  ..., -0.0000,  0.0000,  0.0000],
        [ 2.4096,  0.0000, -0.0000,  ..., -0.0000,  0.3378,  0.9467],
        ...,
        [ 0.0000,  1.2617, -0.0000,  ..., -0.0000,  0.0000,  1.3716],
        [ 0.0000,  0.0000,  0.6051,  ..., -0.5900,  0.0000,  1.3716],
        [ 0.2750,  1.6936,  0.0000,  ..., -0.0000,  1.4915,  1.3716]],
       grad_fn=<MulBackward0>)

### Architecture Description:
The code defines a Feed Forward Neural Network (FFNN) using PyTorch's nn.Module class. The network consists of embedding layers for categorical features, dropout layers, batch normalization for continuous features, and multiple fully connected layers with activation functions.

### Input Handling:
The network takes two types of inputs: categorical features (x_cat) and continuous features (x_cont). Categorical features are embedded using a list of embedding layers, and the embeddings are concatenated. Dropout is applied to the embeddings for regularization.

### Batch Normalization:
Continuous features are normalized using batch normalization (nn.BatchNorm1d). This helps stabilize and accelerate the training process by normalizing the input distributions.

### Fully Connected Layers:
The network contains a series of fully connected layers with ReLU activation, batch normalization, and dropout. These layers progressively transform the concatenated inputs into higher-level representations.

### Output Layer:
The final fully connected layer produces the network's output, which is passed through the layers sequentially to create a multi-layer perceptron (MLP) architecture. The forward method combines categorical and continuous inputs, applies transformations, and produces the output prediction.

In [ ]:
##### Create a Feed Forward Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as F
class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)

        layerlist = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont

        for i in layers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))

        self.layers = nn.Sequential(*layerlist)

    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)

        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

### ReLU (Rectified Linear Unit) is an activation function commonly used in artificial neural networks (ANNs) and deep learning models. It is a non-linear activation function that introduces non-linearity to the network, enabling it to learn complex patterns and relationships in the data

In [ ]:
torch.manual_seed(100)

# Storing the model
model = FeedForwardNN(embedding_dims, len(cont_features), 1, [100, 50], p = 0.4)


In [ ]:
model

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)

nn.MSELoss() stands for Mean Squared Error Loss and is a loss function provided by PyTorch's neural network module (nn). It calculates the mean of the squared differences between predicted values and target values. This loss is often used in regression tasks to measure the dissimilarity between the predicted and actual values, helping to guide the model's parameter updates during training.

In [ ]:
# Define Loss and Optimizers

loss_fun = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
df.shape

(1201, 10)

In [ ]:
# cont_values
cont_values.shape

torch.Size([1201, 5])

In [ ]:
batch_size = 1200
test_size = int(batch_size*0.15)

train_cate = cate_feature[:batch_size - test_size]
test_cate = cate_feature[batch_size - test_size:batch_size]

train_cont = cont_values[:batch_size - test_size]
test_cont = cont_values[batch_size - test_size:batch_size]

y_train = y[:batch_size - test_size]
y_test = y[batch_size - test_size:batch_size]



In [ ]:
len(train_cate), len(test_cate), len(train_cont), len(test_cont), len(y_train), len(y_test)

(1020, 180, 1020, 180, 1020, 180)

In [ ]:
epochs = 5000

final_losses = []

for i in range(epochs):
    i = i+1

    y_pred = model(train_cate, train_cont)

    loss = torch.sqrt(loss_fun(y_pred, y_train))

    final_losses.append(loss)

    if i%10 == 1:
      print("Epoch number: --{}-- and the loss :== {}".format(i,loss.item()))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Epoch number: --1-- and the loss :== 200496.75
Epoch number: --11-- and the loss :== 200493.46875
Epoch number: --21-- and the loss :== 200489.140625
Epoch number: --31-- and the loss :== 200482.640625
Epoch number: --41-- and the loss :== 200473.25
Epoch number: --51-- and the loss :== 200461.375
Epoch number: --61-- and the loss :== 200446.4375
Epoch number: --71-- and the loss :== 200429.421875
Epoch number: --81-- and the loss :== 200407.953125
Epoch number: --91-- and the loss :== 200383.421875
Epoch number: --101-- and the loss :== 200355.234375
Epoch number: --111-- and the loss :== 200322.109375
Epoch number: --121-- and the loss :== 200291.578125
Epoch number: --131-- and the loss :== 200252.0
Epoch number: --141-- and the loss :== 200206.296875
Epoch number: --151-- and the loss :== 200162.9375
Epoch number: --161-- and the loss :== 200112.03125
Epoch number: --171-- and the loss :== 200059.4375
Epoch number: --181-- and the loss :== 200006.109375
Epoch number: --191-- and th

In [ ]:
y_pred = ""

with torch.no_grad():
  y_pred = model(test_cate, test_cont)
  loss = torch.sqrt(loss_fun(y_pred, y_test))

print("RMSE: {}".format(loss))


RMSE: 40477.4609375


In [ ]:
data_verify=pd.DataFrame(y_test.tolist(),columns=["Test"])

In [ ]:
data_predicted=pd.DataFrame(y_pred.tolist(),columns=["Prediction"])


In [ ]:
data_predicted[:10]

,Prediction
0,118166.890625
1,219049.218750
2,127814.625000
3,235170.750000
4,210654.843750
5,205635.062500
6,157286.375000
7,316152.593750
8,150613.578125
9,388799.156250


In [ ]:
final_out = pd.concat([data_verify, data_predicted], axis = 1)

In [ ]:
final_out['Diff'] = final_out['Test'] - final_out['Prediction']

final_out[10:25]

,Test,Prediction,Diff
10,178000.0,216236.984375,-38236.984375
11,186500.0,188186.937500,-1686.937500
12,129500.0,175864.312500,-46364.312500
13,119000.0,110382.234375,8617.765625
14,244000.0,188433.218750,55566.781250
15,130000.0,106684.398438,23315.601562
16,165400.0,238785.343750,-73385.343750
17,127500.0,126735.953125,764.046875
18,301500.0,323178.156250,-21678.156250
19,99900.0,86557.351562,13342.648438


In [ ]:
#### Saving The Model
#### Save the model
torch.save(model,'HousePrice.pt')

In [ ]:
torch.save(model.state_dict(),'HouseWeights.pt')